In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight


In [ ]:
data_dir = "/content/drive/MyDrive/Mastitis_disease_dataset"
symptom_csv = "/content/drive/MyDrive/symptoms.csv"


In [ ]:
img_size = 128
batch_size = 8
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)


In [ ]:
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Found 168 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [ ]:
labels = train_data.classes

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(labels),
    y=labels
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


Class Weights: {0: np.float64(2.625), 1: np.float64(0.6176470588235294)}


In [ ]:
cnn_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

cnn_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(train_data.class_indices)


{'Healthy': 0, 'mastitis': 1}


In [ ]:
class_weights = {
    0: 4.0,  # Healthy (give more importance)
    1: 1.0   # Mastitis
}


In [ ]:
history = cnn_model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    class_weight=class_weights
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 108s 5s/step - accuracy: 0.3287 - loss: 1.1762 - val_accuracy: 0.8293 - val_loss: 0.6364
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 450ms/step - accuracy: 0.6652 - loss: 1.1082 - val_accuracy: 0.8049 - val_loss: 0.6686
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 386ms/step - accuracy: 0.5872 - loss: 1.0720 - val_accuracy: 0.7073 - val_loss: 0.6674
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 10s 350ms/step - accuracy: 0.5872 - loss: 1.0522 - val_accuracy: 0.4878 - val_loss: 0.6954
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 441ms/step - accuracy: 0.5019 - loss: 0.9972 - val_accuracy: 0.7073 - val_loss: 0.6141
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 381ms/step - accuracy: 0.6424 - loss: 1.0653 - val_accuracy: 0.8780 - val_loss: 0.4706
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 414ms/step - accuracy: 0.7914 - loss: 0.9416 - val_accuracy: 0.9024 - val_loss: 0.3853
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 427ms/step - accuracy: 0.6792 - loss: 1.0051 - val_accuracy: 0.

In [ ]:
val_data.reset()
y_true = val_data.classes
y_pred_prob = cnn_model.predict(val_data)
y_pred = (y_pred_prob > 0.5).astype(int).reshape(-1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Healthy", "Mastitis"]))


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 173ms/step

Classification Report:
              precision    recall  f1-score   support

     Healthy       0.47      1.00      0.64         7
    Mastitis       1.00      0.76      0.87        34

    accuracy                           0.80        41
   macro avg       0.73      0.88      0.75        41
weighted avg       0.91      0.80      0.83        41



In [ ]:
cnn_model.save("final_cnn_model1.h5")
print("Model saved successfully")


Model saved successfully


In [ ]:
from google.colab import files
files.download("final_cnn_model1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
data = pd.read_csv(symptom_csv)
data.head()


,Redness,Swelling,Hardness,Fever,LowMilkYield,ClotsInMilk,Label
0,1,1,1,1,1,1,1
1,1,1,0,1,1,1,1
2,1,0,1,1,1,0,1
3,1,1,1,0,1,1,1
4,0,1,1,1,1,1,1


In [ ]:
X = data.drop('Label', axis=1)
y = data['Label']


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
import tensorflow as tf
symptom_model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

symptom_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

symptom_model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 32)             │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 769 (3.00 KB)

 Trainable params: 769 (3.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history2 = symptom_model.fit(
    X_train, y_train,
    epochs=20,
    validation_split=0.2
)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3448 - loss: 0.7306 - precision: 0.4444 - recall: 0.2222 - val_accuracy: 0.3750 - val_loss: 0.7537 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.2414 - loss: 0.7431 - precision: 0.3333 - recall: 0.2222 - val_accuracy: 0.3750 - val_loss: 0.7473 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.5172 - loss: 0.6918 - precision: 0.6250 - recall: 0.5556 - val_accuracy: 0.3750 - val_loss: 0.7411 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.4483 - loss: 0.7201 - precision: 0.5714 - recall: 0.4444 - val_accuracy: 0.3750 - val_loss: 0.7348 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.4828 - loss: 0.7061 - precision: 0.5882 - recall: 0.5556 - val_accuracy: 0.

In [ ]:
y_pred2 = (symptom_model.predict(X_test) > 0.5).astype(int)

print("\nClassification Report:")
print(classification_report(y_test, y_pred2))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.70      1.00      0.82         7

    accuracy                           0.70        10
   macro avg       0.35      0.50      0.41        10
weighted avg       0.49      0.70      0.58        10



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
symptom_model.save("/content/drive/MyDrive/final_symptom_model.h5")


In [ ]:
import os
os.listdir()


['.config', 'drive', 'final_cnn_model1.h5', 'sample_data']

In [ ]:
symptom_model.save("symptom1_model.h5")
print("Symptom model saved!")


Symptom model saved!


In [ ]:
import os
os.listdir()

['.config', 'drive', 'symptom1_model.h5', 'final_cnn_model1.h5', 'sample_data']

In [ ]:
from google.colab import files

files.download("symptom1_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>